# LangChain Expression Language (LCEL)

This example refers to [LangChain開發手冊(旗標)](https://www.tenlong.com.tw/products/9789863127918)

In [54]:
!pip install langchain langchain_openai rich --quiet

In [55]:
# 匯入套件和金鑰
import os
from google.colab import userdata
from rich import print as pprint

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### 簡單使用 LCEL

In [56]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
chat_model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(
    '{city} 位於那一個國家？')

# LCEL
chain = prompt | chat_model | str_parser

In [57]:
print(chain.invoke({"city":"台北"}))

台北是位於中華民國（Taiwan）的首都。


### 手工串接個別物件

In [58]:
content = str_parser.invoke(
    chat_model.invoke(
        prompt.invoke({'city': '台北'})))
print(content)

台北是位於台灣的首都。


In [59]:
class make_chain:
    def __init__(self, runnable_list):
        self.__runnable_list = runnable_list
    def invoke(self, arg):
        for runnable in self.__runnable_list:
            arg = runnable.invoke(arg)
        return arg

find_country_chain = make_chain(
    [prompt, chat_model, str_parser]
)

find_country_chain.invoke({'city': '京都'})

'京都位於日本。'

### 使用 RunnableSequence 類別簡化多層函式的呼叫

In [60]:
from langchain_core.runnables import RunnableSequence

find_country_chain = RunnableSequence(
    prompt,
    chat_model,
    str_parser
)
find_country_chain.invoke({'city': 'New York'})

'New York 位於美國。'

### 使用 LCEL 建立兩個Chain, 有共通參數{city}


In [61]:
find_country_chain = prompt | chat_model | str_parser
find_country_chain.invoke({'city': '巴塞隆納'})

'巴塞隆納位於西班牙。'

In [62]:
lang_template = ChatPromptTemplate.from_template('在{city}講哪一種語言？')
find_lang_chain = lang_template | chat_model | str_parser
find_lang_chain.invoke({'city': '開羅'})

'在開羅，主要使用阿拉伯語。此外，在商業和旅遊領域，也會使用英語。'

### 使用 RunnableParallel 合併相同參數執行並整合2個 Runnable 物件



In [63]:
from langchain_core.runnables import RunnableParallel
find_country_and_lang_chain = RunnableParallel(
    country=find_country_chain,
    lang=find_lang_chain
)
find_country_and_lang_chain.invoke({'city': '開羅'})

{'country': '開羅（Cairo）位於埃及（Egypt）的首都。',
 'lang': '在開羅，人們通常講阿拉伯語。此外，英語也是一種普遍使用的語言，尤其在商業場合和旅遊業中。其他語言，如法語和德語，也可能用於特定的場合或社區中。'}

In [64]:
pprint(find_country_and_lang_chain)

RunnableParallel(
    steps__={
        'country': RunnableSequence(
            first=ChatPromptTemplate(
                input_variables=['city'],
                input_types={},
                partial_variables={},
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['city'],
                            input_types={},
                            partial_variables={},
                            template='{city} 位於那一個國家？'
                        ),
                        additional_kwargs={}
                    )
                ]
            ),
            middle=[
                ChatOpenAI(
                    client=<openai.resources.chat.completions.Completions object at 0x7d359729b6a0>,
                    async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7d359752e050>,
                    root_client=<openai.OpenAI object at 0x7d35972993c0>,
                    root_async_client=<openai.AsyncOpenAI object at 0x7d3597299fc0>,
                    model_kwargs={},
                    openai_api_key=SecretStr('**********')
                )
            ],
            last=StrOutputParser()
        ),
        'lang': RunnableSequence(
            first=ChatPromptTemplate(
                input_variables=['city'],
                input_types={},
                partial_variables={},
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['city'],
                            input_types={},
                            partial_variables={},
                            template='在{city}講哪一種語言？'
                        ),
                        additional_kwargs={}
                    )
                ]
            ),
            middle=[
                ChatOpenAI(
                    client=<openai.resources.chat.completions.Completions object at 0x7d359729b6a0>,
                    async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7d359752e050>,
                    root_client=<openai.OpenAI object at 0x7d35972993c0>,
                    root_async_client=<openai.AsyncOpenAI object at 0x7d3597299fc0>,
                    model_kwargs={},
                    openai_api_key=SecretStr('**********')
                )
            ],
            last=StrOutputParser()
        )
    }
)

In [65]:
find_country_and_lang_chain = RunnableParallel({
    'country': find_country_chain,
    'lang': find_lang_chain
})
find_country_and_lang_chain.invoke({'city': '台南'})

{'country': '台南位於台灣。',
 'lang': '在台南以台語和華語為主要的使用語言。台語是當地的方言，而華語則是普遍使用的官方語言。此外，由於台南是一個歷史悠久的城市，也有一些人會使用一些古老的語言或方言與族群語言。'}

In [66]:
summary_template = ChatPromptTemplate.from_template('{country}{lang}')
summary_chain = (
    {
        'country': find_country_chain,
        'lang': find_lang_chain
    }
    | summary_template)
pprint(summary_chain.invoke({'city': '釜山'}))

ChatPromptValue(
    messages=[
        HumanMessage(
            content='釜山（Busan）位於韓國。在釜山，韓語是主要使用的語言。當地居民通常使用韓語進行日常交流和溝通。
然而，由於釜山是一個熱門的旅遊目的地，因此也可以在一些觀光景點和飯店等地方遇到會說英語的人。',
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

錯誤示範

In [67]:
error_chain = {'key': 'hello'} | summary_template

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

## 3-2 LCEL 實用功能

### | 串接可呼叫物件

In [68]:
from operator import attrgetter
get_messages = attrgetter('messages')
from operator import itemgetter
get_first_item = itemgetter(0)

In [69]:
summary = (summary_chain
           | get_messages
           | get_first_item
           | str_parser)
summary.invoke({'city': '釜山'})

'釜山位於南韓。在釜山，人們主要講韓語。因為釜山位於韓國南部，所以韓語是當地使用最廣泛的語言。此外，也有少數人口會使用英語或其他外語。'

### 使用 RunnablePassthrough

https://python.langchain.com/v0.1/docs/expression_language/primitives/passthrough/


In [70]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

### RunnableBinding

指定額外參數(Ex: bind(stop=[]))

In [71]:
chain = ({"city": RunnablePassthrough()}
         | prompt.bind()
         | chat_model.bind(stop=["台灣","臺灣"])
         | str_parser)
print(chain.invoke("台北"))

台北位於


### OpenAI function_calling

In [72]:
from pydantic import BaseModel, Field
class Search(BaseModel):
    """網路搜尋工具"""
    search_input: str = Field(description="應該要搜尋的關鍵字")

In [73]:
model = chat_model.bind_tools([Search])
pprint(model.kwargs["tools"])

[
    {
        'type': 'function',
        'function': {
            'name': 'Search',
            'description': '網路搜尋工具',
            'parameters': {
                'properties': {'search_input': {'description': '應該要搜尋的關鍵字', 'type': 'string'}},
                'required': ['search_input'],
                'type': 'object'
            }
        }
    }
]

In [74]:
prompt = PromptTemplate.from_template("{city}位於哪個國家?")
chain = ({"city": RunnablePassthrough()}
         | prompt
         | model)
pprint(chain.invoke("台北").tool_calls)

[
    {
        'name': 'Search',
        'args': {'search_input': '台北所在國家'},
        'id': 'call_uwd6scxY0eND3e26pL5moIp8',
        'type': 'tool_call'
    }
]

In [75]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
tools_parser = JsonOutputToolsParser()

In [76]:

chain = ({"city": RunnablePassthrough()}
         | prompt
         | model
         | tools_parser)
pprint(chain.invoke("台北"))

[{'args': {'search_input': '台北位於哪個國家?'}, 'type': 'Search'}]

In [99]:
chat_model.invoke(chain.invoke("台北"))

AIMessage(content='您好，請問您想查詢什麼樣的問題或主題的答案呢？我會盡力幫助您找到所需的信息。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 13, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-91f2182c-8c8f-443f-8098-b67f33cbd559-0', usage_metadata={'input_tokens': 13, 'output_tokens': 55, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 分支與合併

In [97]:
person_template = ChatPromptTemplate.from_template(
    "是誰發明{invention}？")
country_template = ChatPromptTemplate.from_template(
    "{person}來自哪個國家？")

person_chain = ({"invention": RunnablePassthrough()}
              | person_template
              | chat_model
              | str_parser)

person_summary_chain = (
    {"person": person_chain}
    | country_template
    | chat_model
    | str_parser
)

person_summary_chain.invoke("珍珠奶茶")

'珍珠奶茶起源於台灣。'

In [102]:
!pip install grandalf
person_summary_chain.get_graph().print_ascii()

+-----------------------+  
| Parallel<person>Input |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *       

### 零污染計畫書Chain

In [78]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
format_instructions = json_parser.get_format_instructions()

In [79]:
# 制定提示模板
prompt1 = ChatPromptTemplate.from_template(
    "請根據{attribute}特性，推薦一種環保的再生能源。請僅提供能源的名稱："
)
prompt2 = ChatPromptTemplate.from_template(
    "在永續發展中，{energy}能源通常用於製造哪種環保材料？請僅提供能源材料的名稱："
    "{format_instructions}"
)
prompt3 = ChatPromptTemplate.from_template(
    "假設每個國家的能源發展是相等的，哪個國家使用{energy}能源可以做得最好？"
    "請僅提供國家/地區名稱："
)
prompt4 = ChatPromptTemplate.from_template(
    "請結合{material}和{country}，描述一個環境友善的未來生活場景。"
)

prompt2 = prompt2.partial(format_instructions=format_instructions)
# 模型串输出模板
model_parser = chat_model | str_parser

# 能源生成鏈
energy_generator = (
    {"attribute": RunnablePassthrough()}
    | prompt1
    | {"energy": model_parser}
)

# 能源材料
energy_to_material = prompt2 | chat_model | json_parser

# 能源使用做得最好的國家
material_to_country = prompt3 | model_parser

# 结合以上
question_generator = (
    energy_generator
    | {"material": energy_to_material | itemgetter('環保材料'),
       "country": material_to_country}
    | prompt4
)

In [80]:
energy_to_material.invoke({'energy':"太陽能"})

{'環保材料': '太陽能電池板'}

In [81]:
material_to_country.invoke({'energy':"太陽能"})

'澳大利亞'

In [82]:
prompt = question_generator.invoke("零污染")
print(f"最終產生的問題：{prompt.messages[0].content}\n\n"
      f"AI 回答結果：{chat_model.invoke(prompt).content}")

最終產生的問題：請結合太陽能電池板和以色列，描述一個環境友善的未來生活場景。

AI 回答結果：在未來的以色列，每一棟建築物的屋頂都安裝了太陽能電池板，這些電池板吸收太陽能並轉化為電力，為居民提供清潔且可再生的能源。居民們可以將這些電力用於家庭用電，降低對傳統燃煤能源的依賴，同時也減少對環境的負擔。

此外，以色列的城市裡到處都是綠色植被和公共花園，這些綠地不僅美化了城市，還能吸收二氧化碳並釋放氧氣，改善城市空氣品質。居民們可以在這些綠地裡散步、運動或休閒，享受天然氧氣的清新。

在這個環境友善的未來生活場景中，以色列的居民們將生活在一個乾淨、綠色、可持續的城市裡，他們的生活品質將得到顯著提升，同時也保護了地球的環境，為子孫後代留下一個更美好的未來。


In [83]:
!pip install grandalf
question_generator.get_graph().print_ascii()

             +--------------------------+              
             | Parallel<attribute>Input |              
             +--------------------------+              
                           *                           
                           *                           
                           *                           
                    +-------------+                    
                    | Passthrough |                    
                    +-------------+                    
                           *                           
                           *                           
                           *                           
                +--------------------+                 
                | ChatPromptTemplate |                 
                +--------------------+                 
                           *                           
                           *                           
                           *                    

## 3-3 LCEL 函式應用與分支合併

### RunnableLambda

In [84]:
from langchain_core.runnables import RunnableLambda

In [85]:
def commodity(food):
    # 定義每個商店的商品和價格
    items = {
        "熱狗": 50,
        "漢堡": 70,
        "披薩": 100}
    item = items.get(food)
    print(f"{food}價格：{item}")
    return {"price": item}

In [86]:
food=RunnableLambda(commodity)
food.invoke("披薩")

披薩價格：100


{'price': 100}

In [87]:
prompt = ChatPromptTemplate.from_template("我選擇的商品要多少錢？"
                        "數量{number}價錢{price}")
chain = (
    {
        'price':itemgetter("food") | RunnableLambda(commodity),
        'number':itemgetter("number")
    }
    | prompt
    | chat_model
    | str_parser
)
print(chain.invoke({"food":"漢堡", "number":"101"}))

漢堡價格：70
那麼如果您選擇的商品數量為101個，每個商品的價格為70元，那總共需要支付的金額將為7070元。如果您有任何其他問題或需求，請隨時告訴我！我很樂意幫助您。


### RunnableBranch

In [88]:
chain = (
    PromptTemplate.from_template(
        "根據使用者問題作回答, 將問題分為要求命令或是查詢答案。\n"
        "<問題>\n{question}\n</問題>\n"
        "分類:"""
    )
    | chat_model
    | str_parser
)

In [89]:
print(chain.invoke({"question": "立刻使用 Google 搜尋台積電股票"}))
print(chain.invoke({"question": "告訴我什麼是極光"}))

要求命令
查詢答案


In [90]:
order_chain = (
    PromptTemplate.from_template(
        "你不會思考只根據命令做回應, 每次回答開頭都以 '是的, 主人' "
        "回覆命令\n"
        "問題: {question}\n"
        "回覆:"
    )
    | chat_model
)
ask_chain = (
    PromptTemplate.from_template(
        "你只能回答知識性相關問題, 任何要求命令不會照做也不會回答,"
        "每次回答開頭都以 '根據我的知識' 回覆命令\n"
        "問題: {question}"
        "回覆:"
    )
    | chat_model
)
defult_chain = (
    PromptTemplate.from_template(
        "請回答問題:\n"
        "問題: {question}\n"
        "回覆:"
    )
    | chat_model
)

### **自訂命令判斷function**

In [91]:
def route(info):
    if "查詢答案" in info["topic"]:
        return ask_chain
    elif "要求命令" in info["topic"]:
        return order_chain
    else:
        return defult_chain

In [92]:
from langchain_core.runnables import RunnableLambda

full_chain = ({"topic": chain, "question": lambda x: x["question"]}
             | RunnableLambda(route)
             | str_parser)

In [93]:
print(full_chain.invoke({"question": "去幫我買東西"}))
print('- '*10)
print(full_chain.invoke({"question": "北極圈是在緯度多少以上？"}))

是的, 主人。去幫我買東西。
- - - - - - - - - - 
根據我的知識，北極圈是在北緯 66.5度以上。


### 使用 RunnableBranch

In [94]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "查詢答案" in x["topic"], ask_chain),
    (lambda x: "要求命令" in x["topic"], order_chain),
    defult_chain,
)

In [95]:
full_chain = ({"topic": chain, "question": lambda x: x["question"]}
              | branch
              | str_parser)

In [96]:
print(full_chain.invoke({"question": "幫我寫一篇哈利波特小說短評"}))
print('- '*10)
print(full_chain.invoke({"question": "台北101有多高？"}))

是的, 主人。哈利波特系列是一部極具魔幻色彩的小說，講述了一個關於魔法世界的奇幻故事。透過主角哈利波特的成長歷程，讀者可以感受到友情、愛情和勇氣的重要性。作者JK羅琳巧妙地將魔法世界和人類世界相互牽連，展現出神奇的魔法世界和人性的複雜性。整個系列故事情節跌宕起伏，充滿著懸疑和驚喜，吸引了無數讀者為之著迷。是一部不可錯過的經典之作。
- - - - - - - - - - 
根據我的知識，台北101的高度為508公尺。
